# B-PLIS-RAG: Experiment Notebook

This notebook demonstrates the B-PLIS-RAG system for legal domain RAG with ReFT and activation steering.

## Contents
1. Setup and Installation
2. Data Loading
3. Model Loading
4. ReFT Intervention Training
5. Activation Steering
6. RAG Pipeline Demo
7. Evaluation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/B-PLIS-rag/blob/main/notebooks/experiment.ipynb)

## 1. Setup and Installation

In [ ]:
# For Colab: Clone repo and install dependencies
import os
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone https://github.com/your-org/B-PLIS-rag.git
    %cd B-PLIS-rag
    !pip install -q -r requirements.txt
else:
    # Local development - add parent to path
    sys.path.insert(0, os.path.dirname(os.getcwd()))

print("Setup complete!")

In [ ]:
# Import required modules
import torch
import numpy as np
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Import B-PLIS-RAG modules
from src.config import get_config, setup_environment
from src.model_loader import load_model, get_model_info
from src.reft import ReFTIntervention, ReFTTrainer, ReFTHook, verify_intervention
from src.activation_steering import ActivationSteering, compute_faithfulness_metrics
from src.data_handler import LegalBenchRAG, DataHandler
from src.retriever import FAISSRetriever
from src.rag_pipeline import RAGPipeline
from src.evaluator import Evaluator

# Setup environment (seeds, etc.)
setup_environment()
print("Modules imported successfully!")

## 2. Data Loading

Load the LegalBench-RAG dataset with legal documents and benchmarks.

In [ ]:
# Initialize data handler
data_handler = LegalBenchRAG()

# Download dataset (skip if already downloaded)
# Uncomment the next line to download
# data_handler.download()

print("Data handler initialized")

In [ ]:
# Load corpus (use small subset for demo)
documents = data_handler.load_corpus(
    corpus_types=["contractnli"],  # Start with one corpus
    max_docs_per_type=50  # Limit for demo
)

# Show statistics
stats = data_handler.get_corpus_stats()
print(f"\nLoaded {stats['total_documents']} documents")
for corpus, info in stats['corpus_types'].items():
    print(f"  {corpus}: {info['num_documents']} docs, avg length: {info['avg_doc_length']:.0f} chars")

In [ ]:
# Load benchmarks
benchmarks = data_handler.load_benchmarks()

print(f"\nLoaded {len(benchmarks)} benchmarks:")
for name, examples in benchmarks.items():
    print(f"  {name}: {len(examples)} examples")

In [ ]:
# Create conflict examples for training
conflict_examples = data_handler.create_conflict_examples(num_examples=20)

print(f"\nCreated {len(conflict_examples)} conflict examples")

# Show a sample
if conflict_examples:
    sample = conflict_examples[0]
    print(f"\nSample conflict example:")
    print(f"  Query: {sample.query[:100]}...")
    print(f"  Context: {sample.context[:200]}...")
    print(f"  Answer: {sample.context_answer[:100]}...")

## 3. Model Loading

Load the T5-base model with optimizations.

In [ ]:
# Load T5-base model
model, tokenizer = load_model(
    model_name="t5-base",
    device=str(device),
    freeze_params=True,  # Freeze for efficiency
)

# Show model info
info = get_model_info(model)
print(f"\nModel: {info['name']}")
print(f"Hidden size: {info['hidden_size']}")
print(f"Decoder layers: {info['num_decoder_layers']}")
print(f"Total params: {info['total_params']:,}")
print(f"Trainable params: {info['trainable_params']:,}")
print(f"Device: {info['device']}")
print(f"Dtype: {info['dtype']}")

In [ ]:
# Test basic generation
test_prompt = "What is a contract?"

inputs = tokenizer(test_prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Q: {test_prompt}")
print(f"A: {response}")

## 4. ReFT Intervention Training

Train a low-dimensional latent intervention to steer the model toward context-faithful generation.

In [ ]:
# Create ReFT intervention
intervention = ReFTIntervention(
    hidden_size=model.config.d_model,  # 768 for t5-base
    intervention_dim=16,  # Low-dimensional latent
    init_std=0.02,
)
intervention = intervention.to(device)

print(f"\nReFT Intervention:")
print(f"  Intervention dim: {intervention.intervention_dim}")
print(f"  Hidden size: {intervention.hidden_size}")
print(f"  Parameters: {intervention.num_parameters()}")
print(f"  % of model: {100 * intervention.num_parameters() / info['total_params']:.4f}%")

In [ ]:
# Prepare training examples
training_examples = [
    {
        "query": ex.query,
        "context": ex.context,
        "answer": ex.context_answer,
    }
    for ex in conflict_examples[:10]  # Use subset for demo
]

# If no real data, use synthetic examples
if not training_examples:
    training_examples = [
        {
            "query": "What is confidential information?",
            "context": "Confidential Information means any non-public information disclosed by one party.",
            "answer": "Confidential Information means any non-public information disclosed by one party.",
        },
        {
            "query": "What is the term of this agreement?",
            "context": "This Agreement shall be effective for three (3) years from the Effective Date.",
            "answer": "Three years from the Effective Date.",
        },
    ]

print(f"Training on {len(training_examples)} examples")

In [ ]:
# Create trainer
trainer = ReFTTrainer(
    model=model,
    intervention=intervention,
    tokenizer=tokenizer,
    target_layer=6,  # Mid-layer for balanced intervention
    learning_rate=1e-2,
    num_steps=50,  # Steps per example
    device=device,
)

print("Trainer created")

In [ ]:
# Train the intervention
print("Training ReFT intervention...")
results = trainer.train(training_examples, epochs=1, verbose=True)

print(f"\nTraining complete!")
print(f"  Final avg loss: {results['avg_loss']:.4f}")
print(f"  Z norm: {results['z_norm']:.4f}")

In [ ]:
# Verify the intervention changes outputs
verification = verify_intervention(
    model=model,
    tokenizer=tokenizer,
    intervention=intervention,
    target_layer=6,
    test_prompt="What is a breach of contract?",
)

print("\nVerification Results:")
print(f"  Outputs differ: {verification['outputs_differ']}")
print(f"  Verification passed: {verification['verification_passed']}")
print(f"\n  Zero-z output: {verification['text_zero_z'][:100]}...")
print(f"  Non-zero-z output: {verification['text_nonzero_z'][:100]}...")

## 5. Activation Steering

Compute and apply activation steering vectors for context focus.

In [ ]:
# Create activation steerer
steerer = ActivationSteering(
    model=model,
    tokenizer=tokenizer,
    layer=6,  # Same layer as ReFT
    device=device,
)

print("Activation steerer created")

In [ ]:
# Prepare prompts for steering vector computation
positive_prompts = [  # With context
    f"Use context: {ex['context']}\n\nQuestion: {ex['query']}\n\nAnswer:"
    for ex in training_examples[:5]
]

negative_prompts = [  # Without context
    f"Question: {ex['query']}\n\nAnswer:"
    for ex in training_examples[:5]
]

print(f"Computing steering vector from {len(positive_prompts)} prompt pairs...")

In [ ]:
# Compute steering vector
steering_vector = steerer.compute_steering_vector(
    positive_prompts=positive_prompts,
    negative_prompts=negative_prompts,
    normalize=True,
)

print(f"\nSteering vector computed!")
print(f"  Shape: {steering_vector.shape}")
print(f"  Norm: {steering_vector.norm().item():.4f}")

In [ ]:
# Test steering effect
test_context = "The agreement terminates after 24 months from signing."
test_query = "What is the duration of the agreement?"
test_prompt = f"Use context: {test_context}\n\nQuestion: {test_query}\n\nAnswer:"

inputs = tokenizer(test_prompt, return_tensors="pt").to(device)

# Without steering
with torch.no_grad():
    outputs_no_steer = model.generate(**inputs, max_new_tokens=50)
text_no_steer = tokenizer.decode(outputs_no_steer[0], skip_special_tokens=True)

# With steering
with steerer.apply(multiplier=2.0):
    with torch.no_grad():
        outputs_steer = model.generate(**inputs, max_new_tokens=50)
text_steer = tokenizer.decode(outputs_steer[0], skip_special_tokens=True)

print(f"Context: {test_context}")
print(f"Question: {test_query}")
print(f"\nWithout steering: {text_no_steer}")
print(f"With steering: {text_steer}")

## 6. RAG Pipeline Demo

Put it all together with the complete RAG pipeline.

In [ ]:
# Create retriever and index documents
retriever = FAISSRetriever(
    embedding_model="all-MiniLM-L6-v2",
    device=str(device) if torch.cuda.is_available() else "cpu",
)

# Collect all documents
all_docs = []
for docs in documents.values():
    all_docs.extend(docs)

if all_docs:
    print(f"Indexing {len(all_docs)} documents...")
    retriever.index_documents(all_docs)
    print("Indexing complete!")
else:
    print("No documents to index. Using synthetic data for demo.")

In [ ]:
# Create full RAG pipeline
pipeline = RAGPipeline(
    model_name="t5-base",
    use_reft=True,
    use_steering=True,
    reft_layer=6,
    reft_dim=16,
    steering_layer=6,
    steering_multiplier=2.0,
)

# Transfer our trained components
pipeline.reft_intervention = intervention
pipeline.steerer = steerer
pipeline.retriever = retriever

print("RAG pipeline ready!")

In [ ]:
# Test queries
test_queries = [
    "What is confidential information?",
    "Define breach of contract.",
    "What are the termination clauses?",
]

for query in test_queries:
    print("\n" + "="*60)
    print(f"Query: {query}")
    print("="*60)
    
    response = pipeline.query(query, top_k=3)
    
    print(f"\nAnswer: {response.answer}")
    print(f"\nSources:")
    for i, src in enumerate(response.sources[:2], 1):
        print(f"  {i}. [{src['source']}] Score: {src['score']:.3f}")

## 7. Evaluation

Evaluate the pipeline on benchmarks.

In [ ]:
# Create evaluator
evaluator = Evaluator(pipeline, data_handler)

print("Evaluator ready")

In [ ]:
# Run evaluation on a benchmark (use small subset for demo)
benchmark_name = list(benchmarks.keys())[0] if benchmarks else "contractnli"

print(f"Evaluating on {benchmark_name}...")
metrics = evaluator.evaluate_benchmark(
    benchmark_name=benchmark_name,
    max_examples=10,  # Small subset for demo
)

print(f"\n{metrics}")

In [ ]:
# Compare with baseline (no steering)
print("\nComparison: Baseline vs Steered")
print("="*40)

# Baseline
pipeline.use_reft = False
pipeline.use_steering = False
baseline_response = pipeline.query("What is confidential information?", top_k=3)
print(f"Baseline answer: {baseline_response.answer}")

# Steered
pipeline.use_reft = True
pipeline.use_steering = True
steered_response = pipeline.query("What is confidential information?", top_k=3)
print(f"Steered answer: {steered_response.answer}")

In [ ]:
# Save trained intervention
save_path = "checkpoints/reft_experiment.pt"
import os
os.makedirs(os.path.dirname(save_path), exist_ok=True)

trainer.save(save_path)
print(f"Intervention saved to {save_path}")

## Summary

This notebook demonstrated:

1. **Data Loading**: Loading LegalBench-RAG corpus and benchmarks
2. **Model Setup**: T5-base with frozen parameters
3. **ReFT Training**: Low-dimensional latent interventions (~0.01% of model params)
4. **Activation Steering**: Computing and applying steering vectors
5. **RAG Pipeline**: Complete retrieval-augmented generation
6. **Evaluation**: Character-level precision/recall metrics

### Key Findings

- ReFT interventions can steer generation with minimal parameters
- Activation steering provides runtime control over context focus
- Combined approach (ReFT + steering) enhances faithfulness

### Next Steps

- Train on full dataset (500+ examples)
- Ablate over layers and intervention dimensions
- Evaluate on all LegalBench-RAG benchmarks
- Test bilingual capabilities (English + Hindi)